# Title
[]()

In [2]:
const {getCurrentTimestamp, loadJsonFile, saveResponseJson} = require('../src/fileFunctions')
const { Client } = require('@notionhq/client');
const fs = require('fs');

# Make sure the current code works

In [11]:
async function retrievePage(
  pageId, jsonFileName='../data/notion_page', save = false, appendTimestamp = true
) {
  const { Client } = require('@notionhq/client');
  const fs = require('fs');
  let notionApiKey = process.env.notion_secret;
  let notion = new Client({ auth: notionApiKey });
    try {
      const response = await notion.pages.retrieve({ page_id: pageId });
      if (save) {
        await saveResponseJson(response, jsonFileName, appendTimestamp);
      }
      return response;
    } catch (error) {
      console.error(error);
      throw error;
    }
}

/**
 * Parses a page and retrieves specific data from it.
 *
 * @param {string} pageId - The ID of the page to parse.
 * @param {string} [database='tasks'] - The name of the database to retrieve the data from.
 * @return {object} - The parsed data from the page.
 */
async function parsePage(pageId, database='tasks') {
  let multi_select_rollups = [];
  let relations = [];
  try {
      const data = await retrievePage(pageId);
      const parsed_data = {};
      if (database === 'tasks') {
          parsed_data['Name'] = data['properties']['Name']['title'][0]['plain_text'];
          multi_select_rollups = [
              'Project tags', 
          ]
          relations = ['Project', 'Parent-task', 'Sub-tasks'];
      } else if (database === 'projects') {
        
        parsed_data['Name'] = data['properties']['Project name']['title'][0]['plain_text'];
        multi_select_rollups = [];
        relations = [];
      } 
      if (multi_select_rollups.length > 0) {
          for (let i = 0; i < multi_select_rollups.length; i++) {
              const rollup = multi_select_rollups[i];
              parsed_data[rollup] = data['properties'][rollup]['rollup']['array'][0]['multi_select'].map(item => item['name']);
          }            
      }
      if (relations.length > 0) {
          for (let i = 0; i < relations.length; i++) {
              const relation = relations[i];
              const related = data['properties'][relation]['relation'];
              //  Only parse relation if it is greater than 0
              if (related.length === 0) {
                  parsed_data[relation] = null;
              } else {
                  parsed_data[relation] = [];
                  for (let j = 0; j < related.length; j++) {
                      parsed_data[relation].push(related[j]['id']);
                  }
              };
          }
      }
      return parsed_data
  } catch (error) {
      console.log(`Error for database ${database}: ${error}`);
  };
}

/**
* Parses the time tracking data.
*
* @param {Array} data - The time tracking data to be parsed.
* @param {boolean} [save=false] - Indicates whether to save the parsed data.
* @param {string} [jsonFileName='../data/notion_time_tracking_parsed'] - The file name to save the parsed data.
* @param {boolean} [appendTimestamp=true] - Indicates whether to append a timestamp to the file name.
* @return {Object} - The parsed time tracking data.
*/
async function parseTimeTracking(
data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
) {
const parsed_data = {};
const relations_list = ['Tasks'];
const array_types = ['multi_select', 'relation'];
let properties = Object.keys(data[0]['properties']);
const to_ignore = ['Notes', 'Last edited', 'Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 'Start hr', 'Projects', 'Project tag', 'Project (Rollup)'];
properties = properties.filter(item => !to_ignore.includes(item));
console.log(`Parsing...`);

// for (let i = 0; i < 3; i++) {
// for (let i = 0; i < data.length; i++) { 
for (let i = data.length - 1; i >= 0; i--) { // Iterate in reverse to parse oldest records first
  try {
    const item = data[i];
    const id = item['id'];
    const record = {};
    console.log(`\trecord ${i}, id ${id}`);
    record['url'] = item['url'];
    record['created_time'] = item['created_time'];

    for (let j = 0; j < properties.length; j++) {
      const property = properties[j];
      const property_dict = data[i]['properties'][property];
      const property_type = property_dict['type'];

      if (property_type === 'relation') {
        if (property_dict[property_type].length === 0) {
          record[property] = null;
        } else {
          const relationValues = property_dict[property_type];
      
      
          if (property === 'Tasks') {
            const taskProjects = [];
            const taskProjectTags = [];
      
            for (let k = 0; k < relationValues.length; k++) {
              const task_details = await parsePage(relationValues[k]['id'], database='tasks');
              
              const attributes = Object.keys(task_details);
              const attribute_dict = {};
              for (let attribute = 0;attribute < attributes.length; attribute++) {
                const attribute_list = [];
                attribute_list.push(task_details[attributes[attribute]]);
                attribute_dict[attributes[attribute]] = attribute_list;
              };
              const projectList = task_details['Project'];
              const projectArray = [];
              for (let p = 0; p < projectList.length; p++) {
                const projectId = projectList[p];
                console.log(`\t\tProject ${p}, id ${projectId}`)
                const project = await parsePage(projectId, 'projects'); ////
                let project_attributes = Object.keys(project);
                for (let c = 0; c < project_attributes.length; c++) {
                  const attribute_list = [];
                  attribute_list.push(project[project_attributes[c]]);
                  if (project_attributes[c] === 'Name') {
                    project_attributes[c] = 'Project name';
                  };
                  projectArray.push(attribute_list)
                  attribute_dict[project_attributes[c]] = projectArray;
                };
              }
              
              const task_attributes = Object.keys(attribute_dict)
              for (let c = 0; c < task_attributes.length; c++) {
                record[`Task ${task_attributes[c]}`] = attribute_dict[task_attributes[c]]; // task attributes
              };
            }
          }
        }
      } else if (property_type === 'rollup') {
        rollup_type = property_dict[property_type]['type'];

        if (rollup_type === 'array' && property_dict[property_type]['array'].length > 0) {
          const array_type = property_dict[property_type]['array'][0]['type'];

          if (array_type === 'multi_select' || array_type === 'relation') {
            record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
          } else {
            record[property] = null;
          }
        } else {
          record[property] = null;
        }
      } else if (property_type === 'rich_text' || property_type === 'title') {
        if (property_dict[property_type].length > 0) {
          record[property] = property_dict[property_type][0]['text']['content'];
        } else {
          record[property] = null;
        }
      } else if (property_type === 'formula') {
        formula_type = property_dict[property_type]['type'];
        record[property] = property_dict[property_type][formula_type];
      } else if (property_type === 'multi_select') {
        if (property_dict[property_type].length > 0) {
          record[property] = property_dict[property_type].map(item => item);
        } else {
          record[property] = null;
        }
      } else {
        console.log(`\t\tProperty of type ${property_type} was not parsed: ${property}`);
      }
    }

    parsed_data[id] = record;
  } catch (error) {
    console.error(error);
    break;
  }
}

if (save) {
  saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
}

return parsed_data;
}

async function parseTimeTracking(
    data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
  ) {
    const parsed_data = {};
    const relations_list = ['Tasks'];
    const array_types = ['multi_select', 'relation'];
    let properties = Object.keys(data[0]['properties']);
    const to_ignore = ['Notes', 'Last edited', 'Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 'Start hr', 'Projects', 'Project tag', 'Project (Rollup)'];
    properties = properties.filter(item => !to_ignore.includes(item));
    console.log(`Parsing...`);
  
    // for (let i = 0; i < 3; i++) {
    // for (let i = 0; i < data.length; i++) { 
    for (let i = data.length - 1; i >= 0; i--) { // Iterate in reverse to parse oldest records first
      try {
        const item = data[i];
        const id = item['id'];
        const record = {};
        console.log(`\trecord ${i}, id ${id}`);
        record['url'] = item['url'];
        record['created_time'] = item['created_time'];
  
        for (let j = 0; j < properties.length; j++) {
          const property = properties[j];
          const property_dict = data[i]['properties'][property];
          const property_type = property_dict['type'];
  
          if (property_type === 'relation') {
            if (property_dict[property_type].length === 0) {
              record[property] = null;
            } else {
              const relationValues = property_dict[property_type];
          
          
              if (property === 'Tasks') {
                const taskProjects = [];
                const taskProjectTags = [];
          
                for (let k = 0; k < relationValues.length; k++) {
                  const task_details = await parsePage(relationValues[k]['id'], database='tasks');
                  
                  const attributes = Object.keys(task_details);
                  const attribute_dict = {};
                  for (let attribute = 0;attribute < attributes.length; attribute++) {
                    const attribute_list = [];
                    attribute_list.push(task_details[attributes[attribute]]);
                    attribute_dict[attributes[attribute]] = attribute_list;
                  };
                  const projectList = task_details['Project'];
                  const projectArray = [];
                  for (let p = 0; p < projectList.length; p++) {
                    const projectId = projectList[p];
                    console.log(`\t\tProject ${p}, id ${projectId}`)
                    const project = await parsePage(projectId, 'projects');
                    let project_attributes = Object.keys(project);
                    for (let c = 0; c < project_attributes.length; c++) {
                      const attribute_list = [];
                      attribute_list.push(project[project_attributes[c]]);
                      if (project_attributes[c] === 'Name') {
                        project_attributes[c] = 'Project name';
                      };
                      projectArray.push(attribute_list)
                      attribute_dict[project_attributes[c]] = projectArray;
                    };
                  }
                  
                  const task_attributes = Object.keys(attribute_dict)
                  for (let c = 0; c < task_attributes.length; c++) {
                    record[`Task ${task_attributes[c]}`] = attribute_dict[task_attributes[c]]; // task attributes
                  };
                }
              }
            }
          } else if (property_type === 'rollup') {
            rollup_type = property_dict[property_type]['type'];
  
            if (rollup_type === 'array' && property_dict[property_type]['array'].length > 0) {
              const array_type = property_dict[property_type]['array'][0]['type'];
  
              if (array_type === 'multi_select' || array_type === 'relation') {
                record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
              } else {
                record[property] = null;
              }
            } else {
              record[property] = null;
            }
          } else if (property_type === 'rich_text' || property_type === 'title') {
            if (property_dict[property_type].length > 0) {
              record[property] = property_dict[property_type][0]['text']['content'];
            } else {
              record[property] = null;
            }
          } else if (property_type === 'formula') {
            formula_type = property_dict[property_type]['type'];
            record[property] = property_dict[property_type][formula_type];
          } else if (property_type === 'multi_select') {
            if (property_dict[property_type].length > 0) {
              record[property] = property_dict[property_type].map(item => item);
            } else {
              record[property] = null;
            }
          } else {
            console.log(`\t\tProperty of type ${property_type} was not parsed: ${property}`);
          }
        }
  
        parsed_data[id] = record;
      } catch (error) {
        console.error(error);
        break;
      }
    }
  
    if (save) {
      saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
    }
  
    return parsed_data;
  }

var save = false
var jsonFileName = '../data/raw/NewNotionTimeTracking_2024-01-02T18:11:00-08:00'
var parsedJsonFileName = '../data/test_notion_parsing_2024-01-02T18:11:00-08:00'
var data = loadJsonFile(`${jsonFileName}.json`);
var parsedData = parseTimeTracking(
  data, save=save, path.resolve(parsedJsonFileName),
  appendTimestamp = false
)

Loading JSON file: ../data/raw/NewNotionTimeTracking_2024-01-02T18:11:00-08:00.json
Parsing...
	record 78, id 071294c5-0e6d-438b-be30-78e294b4c0ae


		Project 0, id 465e460f-f6e8-49a0-b9eb-0411a742e71c
	record 77, id 3dbc0bec-3421-43d8-8b7c-9f21dec84930
		Project 0, id 0ef94798-823a-4798-b081-bbd4f58be86c
	record 76, id d392826e-a38c-4b48-af02-2965414b6379
		Project 0, id 0ef94798-823a-4798-b081-bbd4f58be86c
	record 75, id 75d7252b-e481-4801-a0da-608cd5fe6f84
		Project 0, id bedda674-2425-4390-a238-c100da992535
	record 74, id 30a39ecb-9c6b-4678-8a1b-4ef63bbe65b3
		Project 0, id caf14c08-d7fb-4a73-80b5-58041af446a2
	record 73, id c9329a17-fa24-49e6-8e54-199539821bd1
		Project 0, id 8a7f0c8d-2e99-40ce-8209-11a91f577177
	record 72, id 380d1fe9-b24f-4aaa-aa48-12751596c599
		Project 0, id dfa60122-de42-401d-9d4d-fb010c170aae
	record 71, id 59b2ff90-34ee-42ed-9775-17d5b682106e
		Project 0, id caf14c08-d7fb-4a73-80b5-58041af446a2
	record 70, id 3172a910-cd7c-4ca6-8c9d-fbf386654ff1
		Project 0, id dfa60122-de42-401d-9d4d-fb010c170aae
	record 69, id 2768aa28-5ff3-4037-9251-62c7558cf324
		Project 0, id 8ae55237-e37d-4c4c-a9f0-38aeb1925987
	r

In [12]:
parsedData

Promise {
  {
    '071294c5-0e6d-438b-be30-78e294b4c0ae': {
      url: 'https://www.notion.so/2024-01-02-18-40-071294c50e6d438bbe3078e294b4c0ae',
      created_time: '2024-01-03T02:40:00.000Z',
      'Task Tag': [Array],
      Flag: null,
      Elapsed: 0.966666666667,
      'Task Name': [Array],
      'Task Project tags': [Array],
      'Task Project': [Array],
      'Task Parent-task': [Array],
      'Task Sub-tasks': [Array],
      'Task Project name': [Array],
      'start time': '18:40',
      'end time': '19:38',
      'Roadmap Item': [],
      Name: '2024-01-02 18:40'
    },
    '3dbc0bec-3421-43d8-8b7c-9f21dec84930': {
      url: 'https://www.notion.so/2024-01-02-19-38-3dbc0bec342143d88b7c9f21dec84930',
      created_time: '2024-01-03T05:03:00.000Z',
      'Task Tag': [Array],
      Flag: null,
      Elapsed: 1.416666666667,
      'Task Name': [Array],
      'Task Project tags': [Array],
      'Task Project': [Array],
      'Task Parent-task': [Array],
      'Task Sub-tasks': [

In [13]:
data

[
  {
    object: 'page',
    id: '576745a6-714c-401b-8469-5a7b1dcd7ad5',
    created_time: '2024-01-08T06:56:00.000Z',
    last_edited_time: '2024-01-08T07:06:00.000Z',
    created_by: { object: 'user', id: 'f4b38e73-d5f6-43c6-99b5-a5cbf1fb60c2' },
    last_edited_by: { object: 'user', id: 'f4b38e73-d5f6-43c6-99b5-a5cbf1fb60c2' },
    cover: null,
    icon: null,
    parent: {
      type: 'database_id',
      database_id: 'bcd0d296-955b-4b92-8621-e4b96ebad29c'
    },
    archived: false,
    properties: {
      'Task Tag': [Object],
      Flag: [Object],
      'Created time': [Object],
      Notes: [Object],
      Elapsed: [Object],
      Projects: [Object],
      'Start min': [Object],
      summary: [Object],
      'End min': [Object],
      'follow up task': [Object],
      URL: [Object],
      'End hr': [Object],
      Tasks: [Object],
      'Last edited': [Object],
      'Project tag': [Object],
      'start time': [Object],
      'Project (Rollup)': [Object],
      'Start hr': [

# Iteration 1

In [3]:
async function retrievePage(
    pageId, jsonFileName='../data/notion_page', save = false, appendTimestamp = true
  ) {
    const { Client } = require('@notionhq/client');
    const fs = require('fs');
    let notionApiKey = process.env.notion_secret;
    let notion = new Client({ auth: notionApiKey });
      try {
        const response = await notion.pages.retrieve({ page_id: pageId });
        if (save) {
          await saveResponseJson(response, jsonFileName, appendTimestamp);
        }
        return response;
      } catch (error) {
        console.error(error);
        throw error;
      }
  }
  
  /**
   * Parses a page and retrieves specific data from it.
   *
   * @param {string} pageId - The ID of the page to parse.
   * @param {string} [database='tasks'] - The name of the database to retrieve the data from.
   * @return {object} - The parsed data from the page.
   */
  async function parsePage(pageId, database='tasks') {
    let multi_select_rollups = [];
    let relations = [];
    try {
        const data = await retrievePage(pageId);
        const parsed_data = {};
        if (database === 'tasks') {
            parsed_data['Name'] = data['properties']['Name']['title'][0]['plain_text'];
            multi_select_rollups = [
                'Project tags', 
            ]
            relations = ['Project', 'Parent-task', 'Sub-tasks'];
        } else if (database === 'projects') {
          
          parsed_data['Name'] = data['properties']['Project name']['title'][0]['plain_text'];
          multi_select_rollups = [];
          relations = [];
        } 
        if (multi_select_rollups.length > 0) {
            for (let i = 0; i < multi_select_rollups.length; i++) {
                const rollup = multi_select_rollups[i];
                parsed_data[rollup] = data['properties'][rollup]['rollup']['array'][0]['multi_select'].map(item => item['name']);
            }            
        }
        if (relations.length > 0) {
            for (let i = 0; i < relations.length; i++) {
                const relation = relations[i];
                const related = data['properties'][relation]['relation'];
                //  Only parse relation if it is greater than 0
                if (related.length === 0) {
                    parsed_data[relation] = null;
                } else {
                    parsed_data[relation] = [];
                    for (let j = 0; j < related.length; j++) {
                        parsed_data[relation].push(related[j]['id']);
                    }
                };
            }
        }
        return parsed_data
    } catch (error) {
        console.log(`Error for database ${database}: ${error}`);
    };
  }
  
  /**
  * Parses the time tracking data.
  *
  * @param {Array} data - The time tracking data to be parsed.
  * @param {boolean} [save=false] - Indicates whether to save the parsed data.
  * @param {string} [jsonFileName='../data/notion_time_tracking_parsed'] - The file name to save the parsed data.
  * @param {boolean} [appendTimestamp=true] - Indicates whether to append a timestamp to the file name.
  * @return {Object} - The parsed time tracking data.
  */
  async function parseTimeTracking(
  data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
  ) {
  const parsed_data = {};
  const relations_list = ['Tasks'];
  const array_types = ['multi_select', 'relation'];
  let properties = Object.keys(data[0]['properties']);
  const to_ignore = ['Notes', 'Last edited', 'Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 'Start hr', 'Projects', 'Project tag', 'Project (Rollup)'];
  properties = properties.filter(item => !to_ignore.includes(item));
  console.log(`Parsing...`);
  
  // for (let i = 0; i < 3; i++) {
  // for (let i = 0; i < data.length; i++) { 
  for (let i = data.length - 1; i >= 0; i--) { // Iterate in reverse to parse oldest records first
    try {
      const item = data[i];
      const id = item['id'];
      const record = {};
      console.log(`\trecord ${i}, id ${id}`);
      record['url'] = item['url'];
      record['created_time'] = item['created_time'];
  
      for (let j = 0; j < properties.length; j++) {
        const property = properties[j];
        const property_dict = data[i]['properties'][property];
        const property_type = property_dict['type'];
  
        if (property_type === 'relation') {
          if (property_dict[property_type].length === 0) {
            record[property] = null;
          } else {
            const relationValues = property_dict[property_type];
        
        
            if (property === 'Tasks') {
              const taskProjects = [];
              const taskProjectTags = [];
        
              for (let k = 0; k < relationValues.length; k++) {
                const task_details = await parsePage(relationValues[k]['id'], database='tasks');
                
                const attributes = Object.keys(task_details);
                const attribute_dict = {};
                for (let attribute = 0;attribute < attributes.length; attribute++) {
                  const attribute_list = [];
                  attribute_list.push(task_details[attributes[attribute]]);
                  attribute_dict[attributes[attribute]] = attribute_list;
                };
                const projectList = task_details['Project'];
                const projectArray = [];
                for (let p = 0; p < projectList.length; p++) {
                  const projectId = projectList[p];
                  console.log(`\t\tProject ${p}, id ${projectId}`)
                  const project = await parsePage(projectId, 'projects');
                  let project_attributes = Object.keys(project);
                  for (let c = 0; c < project_attributes.length; c++) {
                    const attribute_list = [];
                    attribute_list.push(project[project_attributes[c]]);
                    if (project_attributes[c] === 'Name') {
                      project_attributes[c] = 'Project name';
                    };
                    projectArray.push(attribute_list)
                    attribute_dict[project_attributes[c]] = projectArray;
                  };
                }
                
                const task_attributes = Object.keys(attribute_dict)
                for (let c = 0; c < task_attributes.length; c++) {
                  record[`Task ${task_attributes[c]}`] = attribute_dict[task_attributes[c]]; // task attributes
                };
              }
            }
          }
        } else if (property_type === 'rollup') {
          rollup_type = property_dict[property_type]['type'];
  
          if (rollup_type === 'array' && property_dict[property_type]['array'].length > 0) {
            const array_type = property_dict[property_type]['array'][0]['type'];
  
            if (array_type === 'multi_select' || array_type === 'relation') {
              record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
            } else {
              record[property] = null;
            }
          } else {
            record[property] = null;
          }
        } else if (property_type === 'rich_text' || property_type === 'title') {
          if (property_dict[property_type].length > 0) {
            record[property] = property_dict[property_type][0]['text']['content'];
          } else {
            record[property] = null;
          }
        } else if (property_type === 'formula') {
          formula_type = property_dict[property_type]['type'];
          record[property] = property_dict[property_type][formula_type];
        } else if (property_type === 'multi_select') {
          if (property_dict[property_type].length > 0) {
            record[property] = property_dict[property_type].map(item => item);
          } else {
            record[property] = null;
          }
        } else {
          console.log(`\t\tProperty of type ${property_type} was not parsed: ${property}`);
        }
      }
  
      parsed_data[id] = record;
    } catch (error) {
      console.error(error);
      break;
    }
  }
  
  if (save) {
    saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
  }
  
  return parsed_data;
  }
  
  async function parseTimeTracking(
      data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
    ) {
      const parsed_data = {};
      const relations_list = ['Tasks'];
      const array_types = ['multi_select', 'relation'];
      let properties = Object.keys(data[0]['properties']);
      const to_ignore = ['Last edited', 'Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 'Start hr', 'Projects', 'Project tag', 'Project (Rollup)'];
      properties = properties.filter(item => !to_ignore.includes(item));
      console.log(`Parsing...`);
    
      // for (let i = 0; i < 3; i++) {
      // for (let i = 0; i < data.length; i++) { 
      for (let i = data.length - 1; i >= 0; i--) { // Iterate in reverse to parse oldest records first
        try {
          const item = data[i];
          const id = item['id'];
          const record = {};
          console.log(`\trecord ${i}, id ${id}`);
          record['url'] = item['url'];
          record['created_time'] = item['created_time'];
    
          for (let j = 0; j < properties.length; j++) {
            const property = properties[j];
            const property_dict = data[i]['properties'][property];
            const property_type = property_dict['type'];
    
            if (property_type === 'relation') {
              if (property_dict[property_type].length === 0) {
                record[property] = null;
              } else {
                const relationValues = property_dict[property_type];
            
            
                if (property === 'Tasks') {
                  const taskProjects = [];
                  const taskProjectTags = [];
            
                  for (let k = 0; k < relationValues.length; k++) {
                    const task_details = await parsePage(relationValues[k]['id'], database='tasks');
                    
                    const attributes = Object.keys(task_details);
                    const attribute_dict = {};
                    for (let attribute = 0;attribute < attributes.length; attribute++) {
                      const attribute_list = [];
                      attribute_list.push(task_details[attributes[attribute]]);
                      attribute_dict[attributes[attribute]] = attribute_list;
                    };
                    const projectList = task_details['Project'];
                    const projectArray = [];
                    for (let p = 0; p < projectList.length; p++) {
                      const projectId = projectList[p];
                      console.log(`\t\tProject ${p}, id ${projectId}`)
                      const project = await parsePage(projectId, 'projects');
                      let project_attributes = Object.keys(project);
                      for (let c = 0; c < project_attributes.length; c++) {
                        const attribute_list = [];
                        attribute_list.push(project[project_attributes[c]]);
                        if (project_attributes[c] === 'Name') {
                          project_attributes[c] = 'Project name';
                        };
                        projectArray.push(attribute_list)
                        attribute_dict[project_attributes[c]] = projectArray;
                      };
                    }
                    
                    const task_attributes = Object.keys(attribute_dict)
                    for (let c = 0; c < task_attributes.length; c++) {
                      record[`Task ${task_attributes[c]}`] = attribute_dict[task_attributes[c]]; // task attributes
                    };
                  }
                }
              }
            } else if (property_type === 'rollup') {
              rollup_type = property_dict[property_type]['type'];
    
              if (rollup_type === 'array' && property_dict[property_type]['array'].length > 0) {
                const array_type = property_dict[property_type]['array'][0]['type'];
    
                if (array_type === 'multi_select' || array_type === 'relation') {
                  record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
                } else {
                  record[property] = null;
                }
              } else {
                record[property] = null;
              }
            } else if (property_type === 'rich_text' || property_type === 'title') {
              if (property_dict[property_type].length > 0) {
                record[property] = property_dict[property_type][0]['text']['content'];
              } else {
                record[property] = null;
              }
            } else if (property_type === 'formula') {
              formula_type = property_dict[property_type]['type'];
              record[property] = property_dict[property_type][formula_type];
            } else if (property_type === 'multi_select') {
              if (property_dict[property_type].length > 0) {
                record[property] = property_dict[property_type].map(item => item);
              } else {
                record[property] = null;
              }
            } else {
              console.log(`\t\tProperty of type ${property_type} was not parsed: ${property}`);
            }
          }
    
          parsed_data[id] = record;
        } catch (error) {
          console.error(error);
          break;
        }
      }
    
      if (save) {
        saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
      }
    
      return parsed_data;
    }
  
  var save = true
  // var save = false
  var jsonFileName = '../data/raw/test_time_tracking_data'
  var parsedJsonFileName = '../data/parsed_test_time_tracking_data'
  var data = loadJsonFile(`${jsonFileName}.json`);
  var parsedData = parseTimeTracking(
    data, save=save, path.resolve(parsedJsonFileName),
    appendTimestamp = false
  )

Loading JSON file: ../data/raw/test_time_tracking_data.json
Parsing...
	record 1, id e20d416b-41d8-4d00-8671-d225e349b09f


		Project 0, id 8a7f0c8d-2e99-40ce-8209-11a91f577177
	record 0, id 47e6bba3-76f1-41dc-93bb-1a4f0e1da75c
		Project 0, id feb21887-2f05-462f-b1af-760a358e7ca0
Saved response to /home/silvhua/repositories/notion/data/parsed_test_time_tracking_data.json


## 1.1

In [4]:
async function retrievePage(
    pageId, jsonFileName='../data/notion_page', save = false, appendTimestamp = true
  ) {
    const { Client } = require('@notionhq/client');
    const fs = require('fs');
    let notionApiKey = process.env.notion_secret;
    let notion = new Client({ auth: notionApiKey });
      try {
        const response = await notion.pages.retrieve({ page_id: pageId });
        if (save) {
          await saveResponseJson(response, jsonFileName, appendTimestamp);
        }
        return response;
      } catch (error) {
        console.error(error);
        throw error;
      }
  }
  
  /**
   * Parses a page and retrieves specific data from it.
   *
   * @param {string} pageId - The ID of the page to parse.
   * @param {string} [database='tasks'] - The name of the database to retrieve the data from.
   * @return {object} - The parsed data from the page.
   */
  async function parsePage(pageId, database='tasks') {
    let multi_select_rollups = [];
    let relations = [];
    try {
        const data = await retrievePage(pageId);
        const parsed_data = {};
        if (database === 'tasks') {
            parsed_data['Name'] = data['properties']['Name']['title'][0]['plain_text'];
            multi_select_rollups = [
                'Project tags', 
            ]
            relations = ['Project', 'Parent-task', 'Sub-tasks'];
        } else if (database === 'projects') {
          
          parsed_data['Name'] = data['properties']['Project name']['title'][0]['plain_text'];
          multi_select_rollups = [];
          relations = [];
        } 
        if (multi_select_rollups.length > 0) {
            for (let i = 0; i < multi_select_rollups.length; i++) {
                const rollup = multi_select_rollups[i];
                parsed_data[rollup] = data['properties'][rollup]['rollup']['array'][0]['multi_select'].map(item => item['name']);
            }            
        }
        if (relations.length > 0) {
            for (let i = 0; i < relations.length; i++) {
                const relation = relations[i];
                const related = data['properties'][relation]['relation'];
                //  Only parse relation if it is greater than 0
                if (related.length === 0) {
                    parsed_data[relation] = null;
                } else {
                    parsed_data[relation] = [];
                    for (let j = 0; j < related.length; j++) {
                        parsed_data[relation].push(related[j]['id']);
                    }
                };
            }
        }
        return parsed_data
    } catch (error) {
        console.log(`Error for database ${database}: ${error}`);
    };
  }
  
  /**
  * Parses the time tracking data.
  *
  * @param {Array} data - The time tracking data to be parsed.
  * @param {boolean} [save=false] - Indicates whether to save the parsed data.
  * @param {string} [jsonFileName='../data/notion_time_tracking_parsed'] - The file name to save the parsed data.
  * @param {boolean} [appendTimestamp=true] - Indicates whether to append a timestamp to the file name.
  * @return {Object} - The parsed time tracking data.
  */
  async function parseTimeTracking(
  data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
  ) {
  const parsed_data = {};
  const relations_list = ['Tasks'];
  const array_types = ['multi_select', 'relation'];
  let properties = Object.keys(data[0]['properties']);
  const to_ignore = ['Last edited', 'Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 'Start hr', 'Projects', 'Project tag', 'Project (Rollup)'];
  properties = properties.filter(item => !to_ignore.includes(item));
  console.log(`Parsing...`);
  
  // for (let i = 0; i < 3; i++) {
  // for (let i = 0; i < data.length; i++) { 
  for (let i = data.length - 1; i >= 0; i--) { // Iterate in reverse to parse oldest records first
    try {
      const item = data[i];
      const id = item['id'];
      const record = {};
      console.log(`\trecord ${i}, id ${id}`);
      record['url'] = item['url'];
      record['created_time'] = item['created_time'];
  
      for (let j = 0; j < properties.length; j++) {
        const property = properties[j];
        const property_dict = data[i]['properties'][property];
        const property_type = property_dict['type'];
  
        if (property_type === 'relation') {
          if (property_dict[property_type].length === 0) {
            record[property] = null;
          } else {
            const relationValues = property_dict[property_type];
        
        
            if (property === 'Tasks') {
              const taskProjects = [];
              const taskProjectTags = [];
        
              for (let k = 0; k < relationValues.length; k++) {
                const task_details = await parsePage(relationValues[k]['id'], database='tasks');
                
                const attributes = Object.keys(task_details);
                const attribute_dict = {};
                for (let attribute = 0;attribute < attributes.length; attribute++) {
                  const attribute_list = [];
                  attribute_list.push(task_details[attributes[attribute]]);
                  attribute_dict[attributes[attribute]] = attribute_list;
                };
                const projectList = task_details['Project'];
                const projectArray = [];
                for (let p = 0; p < projectList.length; p++) {
                  const projectId = projectList[p];
                  console.log(`\t\tProject ${p}, id ${projectId}`)
                  const project = await parsePage(projectId, 'projects');
                  let project_attributes = Object.keys(project);
                  for (let c = 0; c < project_attributes.length; c++) {
                    const attribute_list = [];
                    attribute_list.push(project[project_attributes[c]]);
                    if (project_attributes[c] === 'Name') {
                      project_attributes[c] = 'Project name';
                    };
                    projectArray.push(attribute_list)
                    attribute_dict[project_attributes[c]] = projectArray;
                  };
                }
                
                const task_attributes = Object.keys(attribute_dict)
                for (let c = 0; c < task_attributes.length; c++) {
                  record[`Task ${task_attributes[c]}`] = attribute_dict[task_attributes[c]]; // task attributes
                };
              }
            }
          }
        } else if (property_type === 'rollup') {
          rollup_type = property_dict[property_type]['type'];
  
          if (rollup_type === 'array' && property_dict[property_type]['array'].length > 0) {
            const array_type = property_dict[property_type]['array'][0]['type'];
  
            if (array_type === 'multi_select' || array_type === 'relation') {
              record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
            } else {
              record[property] = null;
            }
          } else {
            record[property] = null;
          }
        } else if (property_type === 'rich_text' || property_type === 'title') {
          if (property_dict[property_type].length > 0) {
            record[property] = property_dict[property_type][0]['text']['content'];
          } else {
            record[property] = null;
          }
        } else if (property_type === 'formula') {
          formula_type = property_dict[property_type]['type'];
          record[property] = property_dict[property_type][formula_type];
        } else if (property_type === 'multi_select') {
          if (property_dict[property_type].length > 0) {
            record[property] = property_dict[property_type].map(item => item);
          } else {
            record[property] = null;
          }
        } else {
          console.log(`\t\tProperty of type ${property_type} was not parsed: ${property}`);
        }
      }
  
      parsed_data[id] = record;
    } catch (error) {
      console.error(error);
      break;
    }
  }
  
  if (save) {
    saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
  }
  
  return parsed_data;
  }
  
  async function parseTimeTracking(
      data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
    ) {
      const parsed_data = {};
      const relations_list = ['Tasks'];
      const array_types = ['multi_select', 'relation'];
      let properties = Object.keys(data[0]['properties']);
      const to_ignore = ['Last edited', 'Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 'Start hr', 'Projects', 'Project tag', 'Project (Rollup)'];
      properties = properties.filter(item => !to_ignore.includes(item));
      console.log(`Parsing...`);
    
      // for (let i = 0; i < 3; i++) {
      // for (let i = 0; i < data.length; i++) { 
      for (let i = data.length - 1; i >= 0; i--) { // Iterate in reverse to parse oldest records first
        try {
          const item = data[i];
          const id = item['id'];
          const record = {};
          console.log(`\trecord ${i}, id ${id}`);
          record['url'] = item['url'];
          record['created_time'] = item['created_time'];
    
          for (let j = 0; j < properties.length; j++) {
            const property = properties[j];
            const property_dict = data[i]['properties'][property];
            const property_type = property_dict['type'];
    
            if (property_type === 'relation') {
              if (property_dict[property_type].length === 0) {
                record[property] = null;
              } else {
                const relationValues = property_dict[property_type];
            
            
                if (property === 'Tasks') {
                  const taskProjects = [];
                  const taskProjectTags = [];
            
                  for (let k = 0; k < relationValues.length; k++) {
                    const task_details = await parsePage(relationValues[k]['id'], database='tasks');
                    
                    const attributes = Object.keys(task_details);
                    const attribute_dict = {};
                    for (let attribute = 0;attribute < attributes.length; attribute++) {
                      const attribute_list = [];
                      attribute_list.push(task_details[attributes[attribute]]);
                      attribute_dict[attributes[attribute]] = attribute_list;
                    };
                    const projectList = task_details['Project'];
                    const projectArray = [];
                    for (let p = 0; p < projectList.length; p++) {
                      const projectId = projectList[p];
                      console.log(`\t\tProject ${p}, id ${projectId}`)
                      const project = await parsePage(projectId, 'projects');
                      let project_attributes = Object.keys(project);
                      for (let c = 0; c < project_attributes.length; c++) {
                        const attribute_list = [];
                        attribute_list.push(project[project_attributes[c]]);
                        if (project_attributes[c] === 'Name') {
                          project_attributes[c] = 'Project name';
                        };
                        projectArray.push(attribute_list)
                        attribute_dict[project_attributes[c]] = projectArray;
                      };
                    }
                    
                    const task_attributes = Object.keys(attribute_dict)
                    for (let c = 0; c < task_attributes.length; c++) {
                      record[`Task ${task_attributes[c]}`] = attribute_dict[task_attributes[c]]; // task attributes
                    };
                  }
                }
              }
            } else if (property_type === 'rollup') {
              rollup_type = property_dict[property_type]['type'];
    
              if (rollup_type === 'array' && property_dict[property_type]['array'].length > 0) {
                const array_type = property_dict[property_type]['array'][0]['type'];
    
                if (array_type === 'multi_select' || array_type === 'relation') {
                  record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
                } else {
                  record[property] = null;
                }
              } else {
                record[property] = null;
              }
            } else if (property_type === 'rich_text' || property_type === 'title') {
              if (property_dict[property_type].length > 0) {
                record[property] = property_dict[property_type][0]['text']['content'];
              } else {
                record[property] = null;
              }
            } else if (property_type === 'formula') {
              formula_type = property_dict[property_type]['type'];
              record[property] = property_dict[property_type][formula_type];
            } else if (property_type === 'multi_select') {
              if (property_dict[property_type].length > 0) {
                record[property] = property_dict[property_type].map(item => item);
              } else {
                record[property] = null;
              }
            } else {
              console.log(`\t\tProperty of type ${property_type} was not parsed: ${property}`);
            }
          }
    
          parsed_data[id] = record;
        } catch (error) {
          console.error(error);
          break;
        }
      }
    
      if (save) {
        saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
      }
    
      return parsed_data;
    }
  
  var save = true
  // var save = false
  var jsonFileName = '../data/raw/test_time_tracking_data'
  var parsedJsonFileName = '../data/parsed_test_time_tracking_data'
  var data = loadJsonFile(`${jsonFileName}.json`);
  var parsedData = parseTimeTracking(
    data, save=save, path.resolve(parsedJsonFileName),
    appendTimestamp = false
  )

Loading JSON file: ../data/raw/test_time_tracking_data.json
Parsing...
	record 1, id e20d416b-41d8-4d00-8671-d225e349b09f


		Project 0, id 8a7f0c8d-2e99-40ce-8209-11a91f577177
	record 0, id 47e6bba3-76f1-41dc-93bb-1a4f0e1da75c
		Project 0, id feb21887-2f05-462f-b1af-760a358e7ca0
Saved response to /home/silvhua/repositories/notion/data/parsed_test_time_tracking_data.json


## 1.2

In [5]:
async function retrievePage(
    pageId, jsonFileName='../data/notion_page', save = false, appendTimestamp = true
  ) {
    const { Client } = require('@notionhq/client');
    const fs = require('fs');
    let notionApiKey = process.env.notion_secret;
    let notion = new Client({ auth: notionApiKey });
      try {
        const response = await notion.pages.retrieve({ page_id: pageId });
        if (save) {
          await saveResponseJson(response, jsonFileName, appendTimestamp);
        }
        return response;
      } catch (error) {
        console.error(error);
        throw error;
      }
  }
  
  /**
   * Parses a page and retrieves specific data from it.
   *
   * @param {string} pageId - The ID of the page to parse.
   * @param {string} [database='tasks'] - The name of the database to retrieve the data from.
   * @return {object} - The parsed data from the page.
   */
  async function parsePage(pageId, database='tasks') {
    let multi_select_rollups = [];
    let relations = [];
    try {
        const data = await retrievePage(pageId);
        const parsed_data = {};
        if (database === 'tasks') {
            parsed_data['Name'] = data['properties']['Name']['title'][0]['plain_text'];
            multi_select_rollups = [
                'Project tags', 
            ]
            relations = ['Project', 'Parent-task', 'Sub-tasks'];
        } else if (database === 'projects') {
          
          parsed_data['Name'] = data['properties']['Project name']['title'][0]['plain_text'];
          multi_select_rollups = [];
          relations = [];
        } 
        if (multi_select_rollups.length > 0) {
            for (let i = 0; i < multi_select_rollups.length; i++) {
                const rollup = multi_select_rollups[i];
                parsed_data[rollup] = data['properties'][rollup]['rollup']['array'][0]['multi_select'].map(item => item['name']);
            }            
        }
        if (relations.length > 0) {
            for (let i = 0; i < relations.length; i++) {
                const relation = relations[i];
                const related = data['properties'][relation]['relation'];
                //  Only parse relation if it is greater than 0
                if (related.length === 0) {
                    parsed_data[relation] = null;
                } else {
                    parsed_data[relation] = [];
                    for (let j = 0; j < related.length; j++) {
                        parsed_data[relation].push(related[j]['id']);
                    }
                };
            }
        }
        return parsed_data
    } catch (error) {
        console.log(`Error for database ${database}: ${error}`);
    };
  }
  
  /**
  * Parses the time tracking data.
  *
  * @param {Array} data - The time tracking data to be parsed.
  * @param {boolean} [save=false] - Indicates whether to save the parsed data.
  * @param {string} [jsonFileName='../data/notion_time_tracking_parsed'] - The file name to save the parsed data.
  * @param {boolean} [appendTimestamp=true] - Indicates whether to append a timestamp to the file name.
  * @return {Object} - The parsed time tracking data.
  */
  async function parseTimeTracking(
  data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
  ) {
  const parsed_data = {};
  const relations_list = ['Tasks'];
  const array_types = ['multi_select', 'relation'];
  let properties = Object.keys(data[0]['properties']);
  const to_ignore = ['Last edited', 'Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 'Start hr', 'Projects', 'Project tag', 'Project (Rollup)'];
  properties = properties.filter(item => !to_ignore.includes(item));
  console.log(`Parsing...`);
  
  // for (let i = 0; i < 3; i++) {
  // for (let i = 0; i < data.length; i++) { 
  for (let i = data.length - 1; i >= 0; i--) { // Iterate in reverse to parse oldest records first
    try {
      const item = data[i];
      const id = item['id'];
      const record = {};
      console.log(`\trecord ${i}, id ${id}`);
      record['url'] = item['url'];
      record['created_time'] = item['created_time'];
  
      for (let j = 0; j < properties.length; j++) {
        const property = properties[j];
        const property_dict = data[i]['properties'][property];
        const property_type = property_dict['type'];
  
        if (property_type === 'relation') {
          if (property_dict[property_type].length === 0) {
            record[property] = null;
          } else {
            const relationValues = property_dict[property_type];
        
        
            if (property === 'Tasks') {
              const taskProjects = [];
              const taskProjectTags = [];
        
              for (let k = 0; k < relationValues.length; k++) {
                const task_details = await parsePage(relationValues[k]['id'], database='tasks');
                
                const attributes = Object.keys(task_details);
                const attribute_dict = {};
                for (let attribute = 0;attribute < attributes.length; attribute++) {
                  const attribute_list = [];
                  attribute_list.push(task_details[attributes[attribute]]);
                  attribute_dict[attributes[attribute]] = attribute_list;
                };
                const projectList = task_details['Project'];
                const projectArray = [];
                for (let p = 0; p < projectList.length; p++) {
                  const projectId = projectList[p];
                  console.log(`\t\tProject ${p}, id ${projectId}`)
                  const project = await parsePage(projectId, 'projects');
                  let project_attributes = Object.keys(project);
                  for (let c = 0; c < project_attributes.length; c++) {
                    const attribute_list = [];
                    attribute_list.push(project[project_attributes[c]]);
                    if (project_attributes[c] === 'Name') {
                      project_attributes[c] = 'Project name';
                    };
                    projectArray.push(attribute_list)
                    attribute_dict[project_attributes[c]] = projectArray;
                  };
                }
                
                const task_attributes = Object.keys(attribute_dict)
                for (let c = 0; c < task_attributes.length; c++) {
                  record[`Task ${task_attributes[c]}`] = attribute_dict[task_attributes[c]]; // task attributes
                };
              }
            }
          }
        } else if (property_type === 'rollup') {
          rollup_type = property_dict[property_type]['type'];
  
          if (rollup_type === 'array' && property_dict[property_type]['array'].length > 0) {
            const array_type = property_dict[property_type]['array'][0]['type'];
  
            if (array_type === 'multi_select' || array_type === 'relation') {
              record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
            } else {
              record[property] = null;
            }
          } else {
            record[property] = null;
          }
        } else if (property_type === 'rich_text' || property_type === 'title') {
          if (property_dict[property_type].length > 0) {
            record[property] = property_dict[property_type][0]['text']['content'];
          } else {
            record[property] = null;
          }
        } else if (property_type === 'formula') {
          formula_type = property_dict[property_type]['type'];
          record[property] = property_dict[property_type][formula_type];
        } else if (property_type === 'multi_select') {
          if (property_dict[property_type].length > 0) {
            record[property] = property_dict[property_type].map(item => item);
          } else {
            record[property] = null;
          }
        } else {
          console.log(`\t\tProperty of type ${property_type} was not parsed: ${property}`);
        }
      }
  
      parsed_data[id] = record;
    } catch (error) {
      console.error(error);
      break;
    }
  }
  
  if (save) {
    saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
  }
  
  return parsed_data;
  }
  
  async function parseTimeTracking(
      data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
    ) {
      const parsed_data = {};
      const relations_list = ['Tasks'];
      const array_types = ['multi_select', 'relation'];
      let properties = Object.keys(data[0]['properties']);
      const to_ignore = ['Last edited', 'Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 'Start hr', 'Projects', 'Project tag', 'Project (Rollup)'];
      properties = properties.filter(item => !to_ignore.includes(item));
      console.log(`Parsing...`);
    
      // for (let i = 0; i < 3; i++) {
      // for (let i = 0; i < data.length; i++) { 
      for (let i = data.length - 1; i >= 0; i--) { // Iterate in reverse to parse oldest records first
        try {
          const item = data[i];
          const id = item['id'];
          const record = {};
          console.log(`\trecord ${i}, id ${id}`);
          record['url'] = item['url'];
          record['created_time'] = item['created_time'];
    
          for (let j = 0; j < properties.length; j++) {
            const property = properties[j];
            const property_dict = data[i]['properties'][property];
            const property_type = property_dict['type'];
    
            if (property_type === 'relation') {
              if (property_dict[property_type].length === 0) {
                record[property] = null;
              } else {
                const relationValues = property_dict[property_type];
            
            
                if (property === 'Tasks') {
                  const taskProjects = [];
                  const taskProjectTags = [];
            
                  for (let k = 0; k < relationValues.length; k++) {
                    const task_details = await parsePage(relationValues[k]['id'], database='tasks');
                    
                    const attributes = Object.keys(task_details);
                    const attribute_dict = {};
                    for (let attribute = 0;attribute < attributes.length; attribute++) {
                      const attribute_list = [];
                      attribute_list.push(task_details[attributes[attribute]]);
                      attribute_dict[attributes[attribute]] = attribute_list;
                    };
                    const projectList = task_details['Project'];
                    const projectArray = [];
                    for (let p = 0; p < projectList.length; p++) {
                      const projectId = projectList[p];
                      console.log(`\t\tProject ${p}, id ${projectId}`)
                      const project = await parsePage(projectId, 'projects');
                      let project_attributes = Object.keys(project);
                      for (let c = 0; c < project_attributes.length; c++) {
                        const attribute_list = [];
                        attribute_list.push(project[project_attributes[c]]);
                        if (project_attributes[c] === 'Name') {
                          project_attributes[c] = 'Project name';
                        };
                        projectArray.push(attribute_list)
                        attribute_dict[project_attributes[c]] = projectArray;
                      };
                    }
                    
                    const task_attributes = Object.keys(attribute_dict)
                    for (let c = 0; c < task_attributes.length; c++) {
                      record[`Task ${task_attributes[c]}`] = attribute_dict[task_attributes[c]]; // task attributes
                    };
                  }
                }
              }
            } else if (property_type === 'rollup') {
              rollup_type = property_dict[property_type]['type'];
    
              if (rollup_type === 'array' && property_dict[property_type]['array'].length > 0) {
                const array_type = property_dict[property_type]['array'][0]['type'];
    
                if (array_type === 'multi_select' || array_type === 'relation') {
                  record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
                } else {
                  record[property] = null;
                }
              } else {
                record[property] = null;
              }
            } else if (property_type === 'rich_text' || property_type === 'title') {
              if (property_dict[property_type].length > 0) { 
                rich_text_array = property_dict[property_type];
                content_array = [];
                for (let k = 0; k < rich_text_array.length; k++) {
                    content_array.push(rich_text_array[k]['text']['content']);
                }
                record[property] = content_array.join(' ');
              } else {
                record[property] = null;
              }
            } else if (property_type === 'formula') {
              formula_type = property_dict[property_type]['type'];
              record[property] = property_dict[property_type][formula_type];
            } else if (property_type === 'multi_select') {
              if (property_dict[property_type].length > 0) {
                record[property] = property_dict[property_type].map(item => item);
              } else {
                record[property] = null;
              }
            } else {
              console.log(`\t\tProperty of type ${property_type} was not parsed: ${property}`);
            }
          }
    
          parsed_data[id] = record;
        } catch (error) {
          console.error(error);
          break;
        }
      }
    
      if (save) {
        saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
      }
    
      return parsed_data;
    }
  
  var save = true
  // var save = false
  var jsonFileName = '../data/raw/test_time_tracking_data'
  var parsedJsonFileName = '../data/parsed_test_time_tracking_data'
  var data = loadJsonFile(`${jsonFileName}.json`);
  var parsedData = parseTimeTracking(
    data, save=save, path.resolve(parsedJsonFileName),
    appendTimestamp = false
  )

Loading JSON file: ../data/raw/test_time_tracking_data.json
Parsing...
	record 1, id e20d416b-41d8-4d00-8671-d225e349b09f


		Project 0, id 8a7f0c8d-2e99-40ce-8209-11a91f577177
	record 0, id 47e6bba3-76f1-41dc-93bb-1a4f0e1da75c


TypeError: Cannot read properties of undefined (reading 'content')
    at parseTimeTracking (evalmachine.<anonymous>:294:66)
    at processTicksAndRejections (node:internal/process/task_queues:96:5)


Saved response to /home/silvhua/repositories/notion/data/parsed_test_time_tracking_data.json


## 1.21 Able to parse rich text if not a mention 
Next: need to parse mentions, which requires another API call.

In [6]:
async function retrievePage(
    pageId, jsonFileName='../data/notion_page', save = false, appendTimestamp = true
  ) {
    const { Client } = require('@notionhq/client');
    const fs = require('fs');
    let notionApiKey = process.env.notion_secret;
    let notion = new Client({ auth: notionApiKey });
      try {
        const response = await notion.pages.retrieve({ page_id: pageId });
        if (save) {
          await saveResponseJson(response, jsonFileName, appendTimestamp);
        }
        return response;
      } catch (error) {
        console.error(error);
        throw error;
      }
  }
  
  /**
   * Parses a page and retrieves specific data from it.
   *
   * @param {string} pageId - The ID of the page to parse.
   * @param {string} [database='tasks'] - The name of the database to retrieve the data from.
   * @return {object} - The parsed data from the page.
   */
  async function parsePage(pageId, database='tasks') {
    let multi_select_rollups = [];
    let relations = [];
    try {
        const data = await retrievePage(pageId);
        const parsed_data = {};
        if (database === 'tasks') {
            parsed_data['Name'] = data['properties']['Name']['title'][0]['plain_text'];
            multi_select_rollups = [
                'Project tags', 
            ]
            relations = ['Project', 'Parent-task', 'Sub-tasks'];
        } else if (database === 'projects') {
          
          parsed_data['Name'] = data['properties']['Project name']['title'][0]['plain_text'];
          multi_select_rollups = [];
          relations = [];
        } 
        if (multi_select_rollups.length > 0) {
            for (let i = 0; i < multi_select_rollups.length; i++) {
                const rollup = multi_select_rollups[i];
                parsed_data[rollup] = data['properties'][rollup]['rollup']['array'][0]['multi_select'].map(item => item['name']);
            }            
        }
        if (relations.length > 0) {
            for (let i = 0; i < relations.length; i++) {
                const relation = relations[i];
                const related = data['properties'][relation]['relation'];
                //  Only parse relation if it is greater than 0
                if (related.length === 0) {
                    parsed_data[relation] = null;
                } else {
                    parsed_data[relation] = [];
                    for (let j = 0; j < related.length; j++) {
                        parsed_data[relation].push(related[j]['id']);
                    }
                };
            }
        }
        return parsed_data
    } catch (error) {
        console.log(`Error for database ${database}: ${error}`);
    };
  }
  
  /**
  * Parses the time tracking data.
  *
  * @param {Array} data - The time tracking data to be parsed.
  * @param {boolean} [save=false] - Indicates whether to save the parsed data.
  * @param {string} [jsonFileName='../data/notion_time_tracking_parsed'] - The file name to save the parsed data.
  * @param {boolean} [appendTimestamp=true] - Indicates whether to append a timestamp to the file name.
  * @return {Object} - The parsed time tracking data.
  */
  async function parseTimeTracking(
  data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
  ) {
  const parsed_data = {};
  const relations_list = ['Tasks'];
  const array_types = ['multi_select', 'relation'];
  let properties = Object.keys(data[0]['properties']);
  const to_ignore = ['Last edited', 'Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 'Start hr', 'Projects', 'Project tag', 'Project (Rollup)'];
  properties = properties.filter(item => !to_ignore.includes(item));
  console.log(`Parsing...`);
  
  // for (let i = 0; i < 3; i++) {
  // for (let i = 0; i < data.length; i++) { 
  for (let i = data.length - 1; i >= 0; i--) { // Iterate in reverse to parse oldest records first
    try {
      const item = data[i];
      const id = item['id'];
      const record = {};
      console.log(`\trecord ${i}, id ${id}`);
      record['url'] = item['url'];
      record['created_time'] = item['created_time'];
  
      for (let j = 0; j < properties.length; j++) {
        const property = properties[j];
        const property_dict = data[i]['properties'][property];
        const property_type = property_dict['type'];
  
        if (property_type === 'relation') {
          if (property_dict[property_type].length === 0) {
            record[property] = null;
          } else {
            const relationValues = property_dict[property_type];
        
        
            if (property === 'Tasks') {
              const taskProjects = [];
              const taskProjectTags = [];
        
              for (let k = 0; k < relationValues.length; k++) {
                const task_details = await parsePage(relationValues[k]['id'], database='tasks');
                
                const attributes = Object.keys(task_details);
                const attribute_dict = {};
                for (let attribute = 0;attribute < attributes.length; attribute++) {
                  const attribute_list = [];
                  attribute_list.push(task_details[attributes[attribute]]);
                  attribute_dict[attributes[attribute]] = attribute_list;
                };
                const projectList = task_details['Project'];
                const projectArray = [];
                for (let p = 0; p < projectList.length; p++) {
                  const projectId = projectList[p];
                  console.log(`\t\tProject ${p}, id ${projectId}`)
                  const project = await parsePage(projectId, 'projects');
                  let project_attributes = Object.keys(project);
                  for (let c = 0; c < project_attributes.length; c++) {
                    const attribute_list = [];
                    attribute_list.push(project[project_attributes[c]]);
                    if (project_attributes[c] === 'Name') {
                      project_attributes[c] = 'Project name';
                    };
                    projectArray.push(attribute_list)
                    attribute_dict[project_attributes[c]] = projectArray;
                  };
                }
                
                const task_attributes = Object.keys(attribute_dict)
                for (let c = 0; c < task_attributes.length; c++) {
                  record[`Task ${task_attributes[c]}`] = attribute_dict[task_attributes[c]]; // task attributes
                };
              }
            }
          }
        } else if (property_type === 'rollup') {
          rollup_type = property_dict[property_type]['type'];
  
          if (rollup_type === 'array' && property_dict[property_type]['array'].length > 0) {
            const array_type = property_dict[property_type]['array'][0]['type'];
  
            if (array_type === 'multi_select' || array_type === 'relation') {
              record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
            } else {
              record[property] = null;
            }
          } else {
            record[property] = null;
          }
        } else if (property_type === 'rich_text' || property_type === 'title') {
          if (property_dict[property_type].length > 0) {
            record[property] = property_dict[property_type][0]['text']['content'];
          } else {
            record[property] = null;
          }
        } else if (property_type === 'formula') {
          formula_type = property_dict[property_type]['type'];
          record[property] = property_dict[property_type][formula_type];
        } else if (property_type === 'multi_select') {
          if (property_dict[property_type].length > 0) {
            record[property] = property_dict[property_type].map(item => item);
          } else {
            record[property] = null;
          }
        } else {
          console.log(`\t\tProperty of type ${property_type} was not parsed: ${property}`);
        }
      }
  
      parsed_data[id] = record;
    } catch (error) {
      console.error(error);
      break;
    }
  }
  
  if (save) {
    saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
  }
  
  return parsed_data;
  }
  
  async function parseTimeTracking(
      data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
    ) {
      const parsed_data = {};
      const relations_list = ['Tasks'];
      const array_types = ['multi_select', 'relation'];
      let properties = Object.keys(data[0]['properties']);
      const to_ignore = ['Last edited', 'Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 'Start hr', 'Projects', 'Project tag', 'Project (Rollup)'];
      properties = properties.filter(item => !to_ignore.includes(item));
      console.log(`Parsing...`);
    
      // for (let i = 0; i < 3; i++) {
      // for (let i = 0; i < data.length; i++) { 
      for (let i = data.length - 1; i >= 0; i--) { // Iterate in reverse to parse oldest records first
        try {
          const item = data[i];
          const id = item['id'];
          const record = {};
          console.log(`\trecord ${i}, id ${id}`);
          record['url'] = item['url'];
          record['created_time'] = item['created_time'];
    
          for (let j = 0; j < properties.length; j++) {
            const property = properties[j];
            const property_dict = data[i]['properties'][property];
            const property_type = property_dict['type'];
    
            if (property_type === 'relation') {
              if (property_dict[property_type].length === 0) {
                record[property] = null;
              } else {
                const relationValues = property_dict[property_type];
            
            
                if (property === 'Tasks') {
                  const taskProjects = [];
                  const taskProjectTags = [];
            
                  for (let k = 0; k < relationValues.length; k++) {
                    const task_details = await parsePage(relationValues[k]['id'], database='tasks');
                    
                    const attributes = Object.keys(task_details);
                    const attribute_dict = {};
                    for (let attribute = 0;attribute < attributes.length; attribute++) {
                      const attribute_list = [];
                      attribute_list.push(task_details[attributes[attribute]]);
                      attribute_dict[attributes[attribute]] = attribute_list;
                    };
                    const projectList = task_details['Project'];
                    const projectArray = [];
                    for (let p = 0; p < projectList.length; p++) {
                      const projectId = projectList[p];
                      console.log(`\t\tProject ${p}, id ${projectId}`)
                      const project = await parsePage(projectId, 'projects');
                      let project_attributes = Object.keys(project);
                      for (let c = 0; c < project_attributes.length; c++) {
                        const attribute_list = [];
                        attribute_list.push(project[project_attributes[c]]);
                        if (project_attributes[c] === 'Name') {
                          project_attributes[c] = 'Project name';
                        };
                        projectArray.push(attribute_list)
                        attribute_dict[project_attributes[c]] = projectArray;
                      };
                    }
                    
                    const task_attributes = Object.keys(attribute_dict)
                    for (let c = 0; c < task_attributes.length; c++) {
                      record[`Task ${task_attributes[c]}`] = attribute_dict[task_attributes[c]]; // task attributes
                    };
                  }
                }
              }
            } else if (property_type === 'rollup') {
              rollup_type = property_dict[property_type]['type'];
    
              if (rollup_type === 'array' && property_dict[property_type]['array'].length > 0) {
                const array_type = property_dict[property_type]['array'][0]['type'];
    
                if (array_type === 'multi_select' || array_type === 'relation') {
                  record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
                } else {
                  record[property] = null;
                }
              } else {
                record[property] = null;
              }
            } else if (property_type === 'rich_text' || property_type === 'title') {
              if (property_dict[property_type].length > 0) { 
                rich_text_array = property_dict[property_type];
                content_array = [];
                for (let k = 0; k < rich_text_array.length; k++) {
                    rich_text_type = rich_text_array[k]['type'];
                    content_array.push(rich_text_array[k][rich_text_type]['content']);
                }
                record[property] = content_array.join(' ');
              } else {
                record[property] = null;
              }
            } else if (property_type === 'formula') {
              formula_type = property_dict[property_type]['type'];
              record[property] = property_dict[property_type][formula_type];
            } else if (property_type === 'multi_select') {
              if (property_dict[property_type].length > 0) {
                record[property] = property_dict[property_type].map(item => item);
              } else {
                record[property] = null;
              }
            } else {
              console.log(`\t\tProperty of type ${property_type} was not parsed: ${property}`);
            }
          }
    
          parsed_data[id] = record;
        } catch (error) {
          console.error(error);
          break;
        }
      }
    
      if (save) {
        saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
      }
    
      return parsed_data;
    }
  
  var save = true
  // var save = false
  var jsonFileName = '../data/raw/test_time_tracking_data'
  var parsedJsonFileName = '../data/parsed_test_time_tracking_data'
  var data = loadJsonFile(`${jsonFileName}.json`);
  var parsedData = parseTimeTracking(
    data, save=save, path.resolve(parsedJsonFileName),
    appendTimestamp = false
  )

Loading JSON file: ../data/raw/test_time_tracking_data.json
Parsing...
	record 1, id e20d416b-41d8-4d00-8671-d225e349b09f


		Project 0, id 8a7f0c8d-2e99-40ce-8209-11a91f577177
	record 0, id 47e6bba3-76f1-41dc-93bb-1a4f0e1da75c
		Project 0, id feb21887-2f05-462f-b1af-760a358e7ca0
Saved response to /home/silvhua/repositories/notion/data/parsed_test_time_tracking_data.json


# Iteration 2

In [ ]:
async function retrievePage(
    pageId, jsonFileName='../data/notion_page', save = false, appendTimestamp = true
  ) {
    const { Client } = require('@notionhq/client');
    const fs = require('fs');
    let notionApiKey = process.env.notion_secret;
    let notion = new Client({ auth: notionApiKey });
      try {
        const response = await notion.pages.retrieve({ page_id: pageId });
        if (save) {
          await saveResponseJson(response, jsonFileName, appendTimestamp);
        }
        return response;
      } catch (error) {
        console.error(error);
        throw error;
      }
  }
  
  /**
   * Parses a page and retrieves specific data from it.
   *
   * @param {string} pageId - The ID of the page to parse.
   * @param {string} [database='tasks'] - The name of the database to retrieve the data from.
   * @return {object} - The parsed data from the page.
   */
  async function parsePage(pageId, database='tasks') {
    let multi_select_rollups = [];
    let relations = [];
    try {
        const data = await retrievePage(pageId);
        const parsed_data = {};
        if (database === 'tasks') {
            parsed_data['Name'] = data['properties']['Name']['title'][0]['plain_text'];
            multi_select_rollups = [
                'Project tags', 
            ]
            relations = ['Project', 'Parent-task', 'Sub-tasks'];
        } else if (database === 'projects') {
          
          parsed_data['Name'] = data['properties']['Project name']['title'][0]['plain_text'];
          multi_select_rollups = [];
          relations = [];
        } else {
          parsed_data['Name'] = data['properties']['Name']['title'][0]['plain_text'];
          multi_select_rollups = [];
          relations = [];
        }
        if (multi_select_rollups.length > 0) {
            for (let i = 0; i < multi_select_rollups.length; i++) {
                const rollup = multi_select_rollups[i];
                parsed_data[rollup] = data['properties'][rollup]['rollup']['array'][0]['multi_select'].map(item => item['name']);
            }            
        }
        if (relations.length > 0) {
            for (let i = 0; i < relations.length; i++) {
                const relation = relations[i];
                const related = data['properties'][relation]['relation'];
                //  Only parse relation if it is greater than 0
                if (related.length === 0) {
                    parsed_data[relation] = null;
                } else {
                    parsed_data[relation] = [];
                    for (let j = 0; j < related.length; j++) {
                        parsed_data[relation].push(related[j]['id']);
                    }
                };
            }
        }
        return parsed_data
    } catch (error) {
        console.log(`Error for database ${database}: ${error}`);
    };
  }
  
  /**
  * Parses the time tracking data.
  *
  * @param {Array} data - The time tracking data to be parsed.
  * @param {boolean} [save=false] - Indicates whether to save the parsed data.
  * @param {string} [jsonFileName='../data/notion_time_tracking_parsed'] - The file name to save the parsed data.
  * @param {boolean} [appendTimestamp=true] - Indicates whether to append a timestamp to the file name.
  * @return {Object} - The parsed time tracking data.
  */
  async function parseTimeTracking(
  data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
  ) {
  const parsed_data = {};
  const relations_list = ['Tasks'];
  const array_types = ['multi_select', 'relation'];
  let properties = Object.keys(data[0]['properties']);
  const to_ignore = ['Last edited', 'Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 'Start hr', 'Projects', 'Project tag', 'Project (Rollup)'];
  properties = properties.filter(item => !to_ignore.includes(item));
  console.log(`Parsing...`);
  
  // for (let i = 0; i < 3; i++) {
  // for (let i = 0; i < data.length; i++) { 
  for (let i = data.length - 1; i >= 0; i--) { // Iterate in reverse to parse oldest records first
    try {
      const item = data[i];
      const id = item['id'];
      const record = {};
      console.log(`\trecord ${i}, id ${id}`);
      record['url'] = item['url'];
      record['created_time'] = item['created_time'];
  
      for (let j = 0; j < properties.length; j++) {
        const property = properties[j];
        const property_dict = data[i]['properties'][property];
        const property_type = property_dict['type'];
  
        if (property_type === 'relation') {
          if (property_dict[property_type].length === 0) {
            record[property] = null;
          } else {
            const relationValues = property_dict[property_type];
        
        
            if (property === 'Tasks') {
              const taskProjects = [];
              const taskProjectTags = [];
        
              for (let k = 0; k < relationValues.length; k++) {
                const task_details = await parsePage(relationValues[k]['id'], database='tasks');
                
                const attributes = Object.keys(task_details);
                const attribute_dict = {};
                for (let attribute = 0;attribute < attributes.length; attribute++) {
                  const attribute_list = [];
                  attribute_list.push(task_details[attributes[attribute]]);
                  attribute_dict[attributes[attribute]] = attribute_list;
                };
                const projectList = task_details['Project'];
                const projectArray = [];
                for (let p = 0; p < projectList.length; p++) {
                  const projectId = projectList[p];
                  console.log(`\t\tProject ${p}, id ${projectId}`)
                  const project = await parsePage(projectId, 'projects');
                  let project_attributes = Object.keys(project);
                  for (let c = 0; c < project_attributes.length; c++) {
                    const attribute_list = [];
                    attribute_list.push(project[project_attributes[c]]);
                    if (project_attributes[c] === 'Name') {
                      project_attributes[c] = 'Project name';
                    };
                    projectArray.push(attribute_list)
                    attribute_dict[project_attributes[c]] = projectArray;
                  };
                }
                
                const task_attributes = Object.keys(attribute_dict)
                for (let c = 0; c < task_attributes.length; c++) {
                  record[`Task ${task_attributes[c]}`] = attribute_dict[task_attributes[c]]; // task attributes
                };
              }
            }
          }
        } else if (property_type === 'rollup') {
          rollup_type = property_dict[property_type]['type'];
  
          if (rollup_type === 'array' && property_dict[property_type]['array'].length > 0) {
            const array_type = property_dict[property_type]['array'][0]['type'];
  
            if (array_type === 'multi_select' || array_type === 'relation') {
              record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
            } else {
              record[property] = null;
            }
          } else {
            record[property] = null;
          }
        } else if (property_type === 'rich_text' || property_type === 'title') {
          if (property_dict[property_type].length > 0) {
            record[property] = property_dict[property_type][0]['text']['content'];
          } else {
            record[property] = null;
          }
        } else if (property_type === 'formula') {
          formula_type = property_dict[property_type]['type'];
          record[property] = property_dict[property_type][formula_type];
        } else if (property_type === 'multi_select') {
          if (property_dict[property_type].length > 0) {
            record[property] = property_dict[property_type].map(item => item);
          } else {
            record[property] = null;
          }
        } else {
          console.log(`\t\tProperty of type ${property_type} was not parsed: ${property}`);
        }
      }
  
      parsed_data[id] = record;
    } catch (error) {
      console.error(error);
      break;
    }
  }
  
  if (save) {
    saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
  }
  
  return parsed_data;
  }
  
  async function parseTimeTracking(
      data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
    ) {
      const parsed_data = {};
      const relations_list = ['Tasks'];
      const array_types = ['multi_select', 'relation'];
      let properties = Object.keys(data[0]['properties']);
      const to_ignore = ['Last edited', 'Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 'Start hr', 'Projects', 'Project tag', 'Project (Rollup)'];
      properties = properties.filter(item => !to_ignore.includes(item));
      console.log(`Parsing...`);
    
      // for (let i = 0; i < 3; i++) {
      // for (let i = 0; i < data.length; i++) { 
      for (let i = data.length - 1; i >= 0; i--) { // Iterate in reverse to parse oldest records first
        try {
          const item = data[i];
          const id = item['id'];
          const record = {};
          console.log(`\trecord ${i}, id ${id}`);
          record['url'] = item['url'];
          record['created_time'] = item['created_time'];
    
          for (let j = 0; j < properties.length; j++) {
            const property = properties[j];
            const property_dict = data[i]['properties'][property];
            const property_type = property_dict['type'];
    
            if (property_type === 'relation') {
              if (property_dict[property_type].length === 0) {
                record[property] = null;
              } else {
                const relationValues = property_dict[property_type];
            
            
                if (property === 'Tasks') {
                  // const taskProjects = [];
                  // const taskProjectTags = [];
            
                  for (let k = 0; k < relationValues.length; k++) {
                    const task_details = await parsePage(relationValues[k]['id'], database='tasks');
                    
                    const attributes = Object.keys(task_details);
                    const attribute_dict = {};
                    for (let attribute = 0;attribute < attributes.length; attribute++) {
                      const attribute_list = [];
                      attribute_list.push(task_details[attributes[attribute]]);
                      attribute_dict[attributes[attribute]] = attribute_list;
                    };
                    const projectList = task_details['Project'];
                    const projectArray = [];
                    for (let p = 0; p < projectList.length; p++) {
                      const projectId = projectList[p];
                      console.log(`\t\tProject ${p}, id ${projectId}`)
                      const project = await parsePage(projectId, 'projects');
                      let project_attributes = Object.keys(project);
                      for (let c = 0; c < project_attributes.length; c++) {
                        const attribute_list = [];
                        attribute_list.push(project[project_attributes[c]]);
                        if (project_attributes[c] === 'Name') {
                          project_attributes[c] = 'Project name';
                        };
                        projectArray.push(attribute_list)
                        attribute_dict[project_attributes[c]] = projectArray;
                      };
                    }
                    
                    const task_attributes = Object.keys(attribute_dict)
                    for (let c = 0; c < task_attributes.length; c++) {
                      record[`Task ${task_attributes[c]}`] = attribute_dict[task_attributes[c]]; // task attributes
                    };
                  }
                }
              }
            } else if (property_type === 'rollup') {
              rollup_type = property_dict[property_type]['type'];
    
              if (rollup_type === 'array' && property_dict[property_type]['array'].length > 0) {
                const array_type = property_dict[property_type]['array'][0]['type'];
    
                if (array_type === 'multi_select' || array_type === 'relation') {
                  record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
                } else {
                  record[property] = null;
                }
              } else {
                record[property] = null;
              }
            } else if (property_type === 'rich_text' || property_type === 'title') {
              if (property_dict[property_type].length > 0) { 
                rich_text_array = property_dict[property_type];
                content_array = [];
                for (let k = 0; k < rich_text_array.length; k++) {
                    rich_text_type = rich_text_array[k]['type'];
                    if (rich_text_type == 'mention') {
                      mention_type = rich_text_array[k][rich_text_type]['type'];
                      mention_id = rich_text_array[k][rich_text_type][mention_type]['id'];
                      const mention_details = await parsePage(mention_id, database='Unknown');
                      const mention_name = mention_details['Name']
                      content_array.push(mention_name);
                    } else {
                      content_array.push(rich_text_array[k][rich_text_type]['content']);
                    }
                }
                record[property] = content_array.join(' ');
              } else {
                record[property] = null;
              }
            } else if (property_type === 'formula') {
              formula_type = property_dict[property_type]['type'];
              record[property] = property_dict[property_type][formula_type];
            } else if (property_type === 'multi_select') {
              if (property_dict[property_type].length > 0) {
                record[property] = property_dict[property_type].map(item => item);
              } else {
                record[property] = null;
              }
            } else {
              console.log(`\t\tProperty of type ${property_type} was not parsed: ${property}`);
            }
          }
    
          parsed_data[id] = record;
        } catch (error) {
          console.error(error);
          break;
        }
      }
    
      if (save) {
        saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
      }
    
      return parsed_data;
    }
  
  var save = true
  // var save = false
  var jsonFileName = '../data/raw/test_time_tracking_data'
  var parsedJsonFileName = '../data/parsed_test_time_tracking_data'
  var data = loadJsonFile(`${jsonFileName}.json`);
  var parsedData = parseTimeTracking(
    data, save=save, path.resolve(parsedJsonFileName),
    appendTimestamp = false
  )

## 2.1

In [3]:
async function retrievePage(
    pageId, jsonFileName='../data/notion_page', save = false, appendTimestamp = true
  ) {
    const { Client } = require('@notionhq/client');
    const fs = require('fs');
    let notionApiKey = process.env.notion_secret;
    let notion = new Client({ auth: notionApiKey });
      try {
        const response = await notion.pages.retrieve({ page_id: pageId });
        if (save) {
          await saveResponseJson(response, jsonFileName, appendTimestamp);
        }
        return response;
      } catch (error) {
        console.error(error);
        throw error;
      }
  }
  
  /**
   * Parses a page and retrieves specific data from it.
   *
   * @param {string} pageId - The ID of the page to parse.
   * @param {string} [database='tasks'] - The name of the database to retrieve the data from.
   * @return {object} - The parsed data from the page.
   */
  async function parsePage(pageId, database='tasks') {
    let multi_select_rollups = [];
    let relations = [];
    try {
        const data = await retrievePage(pageId);
        const parsed_data = {};
        if (database === 'tasks') {
            parsed_data['Name'] = data['properties']['Name']['title'][0]['plain_text'];
            multi_select_rollups = [
                'Project tags', 
            ]
            relations = ['Project', 'Parent-task', 'Sub-tasks'];
        } else if (database === 'projects') {
          
          parsed_data['Name'] = data['properties']['Project name']['title'][0]['plain_text'];
          multi_select_rollups = [];
          relations = [];
        } else {
          parsed_data['Name'] = data['properties']['Name']['title'][0]['plain_text'];
          multi_select_rollups = [];
          relations = [];
        }
        if (multi_select_rollups.length > 0) {
            for (let i = 0; i < multi_select_rollups.length; i++) {
                const rollup = multi_select_rollups[i];
                parsed_data[rollup] = data['properties'][rollup]['rollup']['array'][0]['multi_select'].map(item => item['name']);
            }            
        }
        if (relations.length > 0) {
            for (let i = 0; i < relations.length; i++) {
                const relation = relations[i];
                const related = data['properties'][relation]['relation'];
                //  Only parse relation if it is greater than 0
                if (related.length === 0) {
                    parsed_data[relation] = null;
                } else {
                    parsed_data[relation] = [];
                    for (let j = 0; j < related.length; j++) {
                        parsed_data[relation].push(related[j]['id']);
                    }
                };
            }
        }
        return parsed_data
    } catch (error) {
        console.log(`Error for database ${database}: ${error}`);
    };
  }
  
  /**
  * Parses the time tracking data.
  *
  * @param {Array} data - The time tracking data to be parsed.
  * @param {boolean} [save=false] - Indicates whether to save the parsed data.
  * @param {string} [jsonFileName='../data/notion_time_tracking_parsed'] - The file name to save the parsed data.
  * @param {boolean} [appendTimestamp=true] - Indicates whether to append a timestamp to the file name.
  * @return {Object} - The parsed time tracking data.
  */
  async function parseTimeTracking(
  data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
  ) {
  const parsed_data = {};
  const relations_list = ['Tasks'];
  const array_types = ['multi_select', 'relation'];
  let properties = Object.keys(data[0]['properties']);
  const to_ignore = ['Last edited', 'Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 'Start hr', 'Projects', 'Project tag', 'Project (Rollup)'];
  properties = properties.filter(item => !to_ignore.includes(item));
  console.log(`Parsing...`);
  
  // for (let i = 0; i < 3; i++) {
  // for (let i = 0; i < data.length; i++) { 
  for (let i = data.length - 1; i >= 0; i--) { // Iterate in reverse to parse oldest records first
    try {
      const item = data[i];
      const id = item['id'];
      const record = {};
      console.log(`\trecord ${i}, id ${id}`);
      record['url'] = item['url'];
      record['created_time'] = item['created_time'];
  
      for (let j = 0; j < properties.length; j++) {
        const property = properties[j];
        const property_dict = data[i]['properties'][property];
        const property_type = property_dict['type'];
  
        if (property_type === 'relation') {
          if (property_dict[property_type].length === 0) {
            record[property] = null;
          } else {
            const relationValues = property_dict[property_type];
        
        
            if (property === 'Tasks') {
              const taskProjects = [];
              const taskProjectTags = [];
        
              for (let k = 0; k < relationValues.length; k++) {
                const task_details = await parsePage(relationValues[k]['id'], database='tasks');
                
                const attributes = Object.keys(task_details);
                const attribute_dict = {};
                for (let attribute = 0;attribute < attributes.length; attribute++) {
                  const attribute_list = [];
                  attribute_list.push(task_details[attributes[attribute]]);
                  attribute_dict[attributes[attribute]] = attribute_list;
                };
                const projectList = task_details['Project'];
                const projectArray = [];
                for (let p = 0; p < projectList.length; p++) {
                  const projectId = projectList[p];
                  console.log(`\t\tProject ${p}, id ${projectId}`)
                  const project = await parsePage(projectId, 'projects');
                  let project_attributes = Object.keys(project);
                  for (let c = 0; c < project_attributes.length; c++) {
                    const attribute_list = [];
                    attribute_list.push(project[project_attributes[c]]);
                    if (project_attributes[c] === 'Name') {
                      project_attributes[c] = 'Project name';
                    };
                    projectArray.push(attribute_list)
                    attribute_dict[project_attributes[c]] = projectArray;
                  };
                }
                
                const task_attributes = Object.keys(attribute_dict)
                for (let c = 0; c < task_attributes.length; c++) {
                  record[`Task ${task_attributes[c]}`] = attribute_dict[task_attributes[c]]; // task attributes
                };
              }
            }
          }
        } else if (property_type === 'rollup') {
          rollup_type = property_dict[property_type]['type'];
  
          if (rollup_type === 'array' && property_dict[property_type]['array'].length > 0) {
            const array_type = property_dict[property_type]['array'][0]['type'];
  
            if (array_type === 'multi_select' || array_type === 'relation') {
              record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
            } else {
              record[property] = null;
            }
          } else {
            record[property] = null;
          }
        } else if (property_type === 'rich_text' || property_type === 'title') {
          if (property_dict[property_type].length > 0) {
            record[property] = property_dict[property_type][0]['text']['content'];
          } else {
            record[property] = null;
          }
        } else if (property_type === 'formula') {
          formula_type = property_dict[property_type]['type'];
          record[property] = property_dict[property_type][formula_type];
        } else if (property_type === 'multi_select') {
          if (property_dict[property_type].length > 0) {
            record[property] = property_dict[property_type].map(item => item);
          } else {
            record[property] = null;
          }
        } else {
          console.log(`\t\tProperty of type ${property_type} was not parsed: ${property}`);
        }
      }
  
      parsed_data[id] = record;
    } catch (error) {
      console.error(error);
      break;
    }
  }
  
  if (save) {
    saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
  }
  
  return parsed_data;
  }
  
  async function parseTimeTracking(
      data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
    ) {
      const parsed_data = {};
      const relations_list = ['Tasks'];
      const array_types = ['multi_select', 'relation'];
      let properties = Object.keys(data[0]['properties']);
      const to_ignore = ['Last edited', 'Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 'Start hr', 'Projects', 'Project tag', 'Project (Rollup)'];
      properties = properties.filter(item => !to_ignore.includes(item));
      console.log(`Parsing...`);
    
      // for (let i = 0; i < 3; i++) {
      // for (let i = 0; i < data.length; i++) { 
      for (let i = data.length - 1; i >= 0; i--) { // Iterate in reverse to parse oldest records first
        try {
          const item = data[i];
          const id = item['id'];
          const record = {};
          console.log(`\trecord ${i}, id ${id}`);
          record['url'] = item['url'];
          record['created_time'] = item['created_time'];
    
          for (let j = 0; j < properties.length; j++) {
            const property = properties[j];
            const property_dict = data[i]['properties'][property];
            const property_type = property_dict['type'];
    
            if (property_type === 'relation') {
              if (property_dict[property_type].length === 0) {
                record[property] = null;
              } else {
                const relationValues = property_dict[property_type];
            
            
                if (property === 'Tasks') {
                  // const taskProjects = [];
                  // const taskProjectTags = [];
            
                  for (let k = 0; k < relationValues.length; k++) {
                    const task_details = await parsePage(relationValues[k]['id'], database='tasks');
                    
                    const attributes = Object.keys(task_details);
                    const attribute_dict = {};
                    for (let attribute = 0;attribute < attributes.length; attribute++) {
                      const attribute_list = [];
                      attribute_list.push(task_details[attributes[attribute]]);
                      attribute_dict[attributes[attribute]] = attribute_list;
                    };
                    const projectList = task_details['Project'];
                    const projectArray = [];
                    for (let p = 0; p < projectList.length; p++) {
                      const projectId = projectList[p];
                      console.log(`\t\tProject ${p}, id ${projectId}`)
                      const project = await parsePage(projectId, 'projects');
                      let project_attributes = Object.keys(project);
                      for (let c = 0; c < project_attributes.length; c++) {
                        const attribute_list = [];
                        attribute_list.push(project[project_attributes[c]]);
                        if (project_attributes[c] === 'Name') {
                          project_attributes[c] = 'Project name';
                        };
                        projectArray.push(attribute_list)
                        attribute_dict[project_attributes[c]] = projectArray;
                      };
                    }
                    
                    const task_attributes = Object.keys(attribute_dict)
                    for (let c = 0; c < task_attributes.length; c++) {
                      record[`Task ${task_attributes[c]}`] = attribute_dict[task_attributes[c]]; // task attributes
                    };
                  }
                }
              }
            } else if (property_type === 'rollup') {
              rollup_type = property_dict[property_type]['type'];
    
              if (rollup_type === 'array' && property_dict[property_type]['array'].length > 0) {
                const array_type = property_dict[property_type]['array'][0]['type'];
    
                if (array_type === 'multi_select' || array_type === 'relation') {
                  record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
                } else {
                  record[property] = null;
                }
              } else {
                record[property] = null;
              }
            } else if (property_type === 'rich_text' || property_type === 'title') {
              if (property_dict[property_type].length > 0) { 
                rich_text_array = property_dict[property_type];
                content_array = [];
                for (let k = 0; k < rich_text_array.length; k++) {
                    rich_text_type = rich_text_array[k]['type'];
                    if (rich_text_type == 'mention') {
                      mention_type = rich_text_array[k][rich_text_type]['type'];
                      mention_id = rich_text_array[k][rich_text_type][mention_type]['id'];
                      const mention_details = await parsePage(mention_id, database='Unknown');
                      const mention_name = mention_details['Name']
                      content_array.push(mention_name);
                    } else {
                      content_array.push(rich_text_array[k][rich_text_type]['content']);
                    }
                }
                record[property] = content_array.join(' ');
              } else {
                record[property] = null;
              }
            } else if (property_type === 'formula') {
              formula_type = property_dict[property_type]['type'];
              record[property] = property_dict[property_type][formula_type];
            } else if (property_type === 'multi_select') {
              if (property_dict[property_type].length > 0) {
                record[property] = property_dict[property_type].map(item => item);
              } else {
                record[property] = null;
              }
            } else {
              console.log(`\t\tProperty of type ${property_type} was not parsed: ${property}`);
            }
          }
    
          parsed_data[id] = record;
        } catch (error) {
          console.error(error);
          break;
        }
      }
    
      if (save) {
        saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
      }
    
      return parsed_data;
    }
  
  var save = true
  // var save = false
  var jsonFileName = '../data/raw/test_time_tracking_data'
  var parsedJsonFileName = '../data/parsed_test_time_tracking_data'
  var data = loadJsonFile(`${jsonFileName}.json`);
  var parsedData = parseTimeTracking(
    data, save=save, path.resolve(parsedJsonFileName),
    appendTimestamp = false
  )

Loading JSON file: ../data/raw/test_time_tracking_data.json
Parsing...
	record 2, id c6bd7066-82f4-4e85-ba23-5db2eea7c6ea


		Project 0, id a4ecf100-912a-422f-8ad6-15b903edd579
	record 1, id e20d416b-41d8-4d00-8671-d225e349b09f
		Project 0, id 8a7f0c8d-2e99-40ce-8209-11a91f577177
	record 0, id 47e6bba3-76f1-41dc-93bb-1a4f0e1da75c


@notionhq/client warn: request fail {
  code: 'object_not_found',
  message: 'Could not find page with ID: 417e1661-087a-4d58-8180-a3f2af85030e. Make sure the relevant pages and databases are shared with your integration.'
}
APIResponseError: Could not find page with ID: 417e1661-087a-4d58-8180-a3f2af85030e. Make sure the relevant pages and databases are shared with your integration.
    at buildRequestError (/home/silvhua/repositories/notion/node_modules/@notionhq/client/build/src/errors.js:162:16)
    at Client.request (/home/silvhua/repositories/notion/node_modules/@notionhq/client/build/src/Client.js:378:54)
    at processTicksAndRejections (node:internal/process/task_queues:96:5)
    at async retrievePage (evalmachine.<anonymous>:9:26)
    at async parsePage (evalmachine.<anonymous>:31:22)
    at async parseTimeTracking (evalmachine.<anonymous>:302:47) {
  code: 'object_not_found',
  status: 404,
  headers: Headers {
    [Symbol(map)]: [Object: null prototype] {
      date: [Array

Error for database Unknown: APIResponseError: Could not find page with ID: 417e1661-087a-4d58-8180-a3f2af85030e. Make sure the relevant pages and databases are shared with your integration.


TypeError: Cannot read properties of undefined (reading 'Name')
    at parseTimeTracking (evalmachine.<anonymous>:303:59)
    at processTicksAndRejections (node:internal/process/task_queues:96:5)


Saved response to /home/silvhua/repositories/notion/data/parsed_test_time_tracking_data.json


## 2.2

In [4]:
async function retrievePage(
    pageId, jsonFileName='../data/notion_page', save = false, appendTimestamp = true
  ) {
    const { Client } = require('@notionhq/client');
    const fs = require('fs');
    let notionApiKey = process.env.notion_secret;
    let notion = new Client({ auth: notionApiKey });
      try {
        const response = await notion.pages.retrieve({ page_id: pageId });
        if (save) {
          await saveResponseJson(response, jsonFileName, appendTimestamp);
        }
        return response;
      } catch (error) {
        console.error(error);
        throw error;
      }
  }
  
  /**
   * Parses a page and retrieves specific data from it.
   *
   * @param {string} pageId - The ID of the page to parse.
   * @param {string} [database='tasks'] - The name of the database to retrieve the data from.
   * @return {object} - The parsed data from the page.
   */
  async function parsePage(pageId, database='tasks') {
    let multi_select_rollups = [];
    let relations = [];
    try {
        const data = await retrievePage(pageId);
        const parsed_data = {};
        if (database === 'tasks') {
            parsed_data['Name'] = data['properties']['Name']['title'][0]['plain_text'];
            multi_select_rollups = [
                'Project tags', 
            ]
            relations = ['Project', 'Parent-task', 'Sub-tasks'];
        } else if (database === 'projects') {
          
          parsed_data['Name'] = data['properties']['Project name']['title'][0]['plain_text'];
          multi_select_rollups = [];
          relations = [];
        } else {
          parsed_data['Name'] = data['properties']['Name']['title'][0]['plain_text'];
          multi_select_rollups = [];
          relations = [];
        }
        if (multi_select_rollups.length > 0) {
            for (let i = 0; i < multi_select_rollups.length; i++) {
                const rollup = multi_select_rollups[i];
                parsed_data[rollup] = data['properties'][rollup]['rollup']['array'][0]['multi_select'].map(item => item['name']);
            }            
        }
        if (relations.length > 0) {
            for (let i = 0; i < relations.length; i++) {
                const relation = relations[i];
                const related = data['properties'][relation]['relation'];
                //  Only parse relation if it is greater than 0
                if (related.length === 0) {
                    parsed_data[relation] = null;
                } else {
                    parsed_data[relation] = [];
                    for (let j = 0; j < related.length; j++) {
                        parsed_data[relation].push(related[j]['id']);
                    }
                };
            }
        }
        return parsed_data
    } catch (error) {
        console.log(`Error for database ${database}: ${error}`);
    };
  }
  
  /**
  * Parses the time tracking data.
  *
  * @param {Array} data - The time tracking data to be parsed.
  * @param {boolean} [save=false] - Indicates whether to save the parsed data.
  * @param {string} [jsonFileName='../data/notion_time_tracking_parsed'] - The file name to save the parsed data.
  * @param {boolean} [appendTimestamp=true] - Indicates whether to append a timestamp to the file name.
  * @return {Object} - The parsed time tracking data.
  */
  async function parseTimeTracking(
  data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
  ) {
  const parsed_data = {};
  const relations_list = ['Tasks'];
  const array_types = ['multi_select', 'relation'];
  let properties = Object.keys(data[0]['properties']);
  const to_ignore = ['Last edited', 'Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 'Start hr', 'Projects', 'Project tag', 'Project (Rollup)'];
  properties = properties.filter(item => !to_ignore.includes(item));
  console.log(`Parsing...`);
  
  // for (let i = 0; i < 3; i++) {
  // for (let i = 0; i < data.length; i++) { 
  for (let i = data.length - 1; i >= 0; i--) { // Iterate in reverse to parse oldest records first
    try {
      const item = data[i];
      const id = item['id'];
      const record = {};
      console.log(`\trecord ${i}, id ${id}`);
      record['url'] = item['url'];
      record['created_time'] = item['created_time'];
  
      for (let j = 0; j < properties.length; j++) {
        const property = properties[j];
        const property_dict = data[i]['properties'][property];
        const property_type = property_dict['type'];
  
        if (property_type === 'relation') {
          if (property_dict[property_type].length === 0) {
            record[property] = null;
          } else {
            const relationValues = property_dict[property_type];
        
        
            if (property === 'Tasks') {
              const taskProjects = [];
              const taskProjectTags = [];
        
              for (let k = 0; k < relationValues.length; k++) {
                const task_details = await parsePage(relationValues[k]['id'], database='tasks');
                
                const attributes = Object.keys(task_details);
                const attribute_dict = {};
                for (let attribute = 0;attribute < attributes.length; attribute++) {
                  const attribute_list = [];
                  attribute_list.push(task_details[attributes[attribute]]);
                  attribute_dict[attributes[attribute]] = attribute_list;
                };
                const projectList = task_details['Project'];
                const projectArray = [];
                for (let p = 0; p < projectList.length; p++) {
                  const projectId = projectList[p];
                  console.log(`\t\tProject ${p}, id ${projectId}`)
                  const project = await parsePage(projectId, 'projects');
                  let project_attributes = Object.keys(project);
                  for (let c = 0; c < project_attributes.length; c++) {
                    const attribute_list = [];
                    attribute_list.push(project[project_attributes[c]]);
                    if (project_attributes[c] === 'Name') {
                      project_attributes[c] = 'Project name';
                    };
                    projectArray.push(attribute_list)
                    attribute_dict[project_attributes[c]] = projectArray;
                  };
                }
                
                const task_attributes = Object.keys(attribute_dict)
                for (let c = 0; c < task_attributes.length; c++) {
                  record[`Task ${task_attributes[c]}`] = attribute_dict[task_attributes[c]]; // task attributes
                };
              }
            }
          }
        } else if (property_type === 'rollup') {
          rollup_type = property_dict[property_type]['type'];
  
          if (rollup_type === 'array' && property_dict[property_type]['array'].length > 0) {
            const array_type = property_dict[property_type]['array'][0]['type'];
  
            if (array_type === 'multi_select' || array_type === 'relation') {
              record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
            } else {
              record[property] = null;
            }
          } else {
            record[property] = null;
          }
        } else if (property_type === 'rich_text' || property_type === 'title') {
          if (property_dict[property_type].length > 0) {
            record[property] = property_dict[property_type][0]['text']['content'];
          } else {
            record[property] = null;
          }
        } else if (property_type === 'formula') {
          formula_type = property_dict[property_type]['type'];
          record[property] = property_dict[property_type][formula_type];
        } else if (property_type === 'multi_select') {
          if (property_dict[property_type].length > 0) {
            record[property] = property_dict[property_type].map(item => item);
          } else {
            record[property] = null;
          }
        } else {
          console.log(`\t\tProperty of type ${property_type} was not parsed: ${property}`);
        }
      }
  
      parsed_data[id] = record;
    } catch (error) {
      console.error(error);
      break;
    }
  }
  
  if (save) {
    saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
  }
  
  return parsed_data;
  }
  
  async function parseTimeTracking(
      data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
    ) {
      const parsed_data = {};
      const relations_list = ['Tasks'];
      const array_types = ['multi_select', 'relation'];
      let properties = Object.keys(data[0]['properties']);
      const to_ignore = ['Last edited', 'Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 'Start hr', 'Projects', 'Project tag', 'Project (Rollup)'];
      properties = properties.filter(item => !to_ignore.includes(item));
      console.log(`Parsing...`);
    
      // for (let i = 0; i < 3; i++) {
      // for (let i = 0; i < data.length; i++) { 
      for (let i = data.length - 1; i >= 0; i--) { // Iterate in reverse to parse oldest records first
        try {
          const item = data[i];
          const id = item['id'];
          const record = {};
          console.log(`\trecord ${i}, id ${id}`);
          record['url'] = item['url'];
          record['created_time'] = item['created_time'];
    
          for (let j = 0; j < properties.length; j++) {
            const property = properties[j];
            const property_dict = data[i]['properties'][property];
            const property_type = property_dict['type'];
    
            if (property_type === 'relation') {
              if (property_dict[property_type].length === 0) {
                record[property] = null;
              } else {
                const relationValues = property_dict[property_type];
            
            
                if (property === 'Tasks') {
                  // const taskProjects = [];
                  // const taskProjectTags = [];
            
                  for (let k = 0; k < relationValues.length; k++) {
                    const task_details = await parsePage(relationValues[k]['id'], database='tasks');
                    
                    const attributes = Object.keys(task_details);
                    const attribute_dict = {};
                    for (let attribute = 0;attribute < attributes.length; attribute++) {
                      const attribute_list = [];
                      attribute_list.push(task_details[attributes[attribute]]);
                      attribute_dict[attributes[attribute]] = attribute_list;
                    };
                    const projectList = task_details['Project'];
                    const projectArray = [];
                    for (let p = 0; p < projectList.length; p++) {
                      const projectId = projectList[p];
                      console.log(`\t\tProject ${p}, id ${projectId}`)
                      const project = await parsePage(projectId, 'projects');
                      let project_attributes = Object.keys(project);
                      for (let c = 0; c < project_attributes.length; c++) {
                        const attribute_list = [];
                        attribute_list.push(project[project_attributes[c]]);
                        if (project_attributes[c] === 'Name') {
                          project_attributes[c] = 'Project name';
                        };
                        projectArray.push(attribute_list)
                        attribute_dict[project_attributes[c]] = projectArray;
                      };
                    }
                    
                    const task_attributes = Object.keys(attribute_dict)
                    for (let c = 0; c < task_attributes.length; c++) {
                      record[`Task ${task_attributes[c]}`] = attribute_dict[task_attributes[c]]; // task attributes
                    };
                  }
                }
              }
            } else if (property_type === 'rollup') {
              rollup_type = property_dict[property_type]['type'];
    
              if (rollup_type === 'array' && property_dict[property_type]['array'].length > 0) {
                const array_type = property_dict[property_type]['array'][0]['type'];
    
                if (array_type === 'multi_select' || array_type === 'relation') {
                  record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
                } else {
                  record[property] = null;
                }
              } else {
                record[property] = null;
              }
            } else if (property_type === 'rich_text' || property_type === 'title') {
              if (property_dict[property_type].length > 0) { 
                rich_text_array = property_dict[property_type];
                content_array = [];
                for (let k = 0; k < rich_text_array.length; k++) {
                    rich_text_type = rich_text_array[k]['type'];
                    if (rich_text_type == 'mention') {
                      mention_type = rich_text_array[k][rich_text_type]['type'];
                      mention_id = rich_text_array[k][rich_text_type][mention_type]['id'];
                      const mention_details = await parsePage(mention_id, database='Unknown');
                      const mention_name = mention_details['Name']
                      content_array.push(mention_name);
                    } else {
                      content_array.push(rich_text_array[k][rich_text_type]['content']);
                    }
                }
                record[property] = content_array.join(' ');
              } else {
                record[property] = null;
              }
            } else if (property_type === 'formula') {
              formula_type = property_dict[property_type]['type'];
              record[property] = property_dict[property_type][formula_type];
            } else if (property_type === 'multi_select') {
              if (property_dict[property_type].length > 0) {
                record[property] = property_dict[property_type].map(item => item);
              } else {
                record[property] = null;
              }
            } else {
              console.log(`\t\tProperty of type ${property_type} was not parsed: ${property}`);
            }
          }
    
          parsed_data[id] = record;
        } catch (error) {
          console.error(error);
          break;
        }
      }
    
      if (save) {
        saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
      }
    
      return parsed_data;
    }
  
  var save = true
  // var save = false
  var jsonFileName = '../data/raw/test_time_tracking_data'
  var parsedJsonFileName = '../data/parsed_test_time_tracking_data'
  var data = loadJsonFile(`${jsonFileName}.json`);
  var parsedData = parseTimeTracking(
    data, save=save, path.resolve(parsedJsonFileName),
    appendTimestamp = false
  )

Loading JSON file: ../data/raw/test_time_tracking_data.json
Parsing...
	record 2, id c6bd7066-82f4-4e85-ba23-5db2eea7c6ea


		Project 0, id a4ecf100-912a-422f-8ad6-15b903edd579
	record 1, id e20d416b-41d8-4d00-8671-d225e349b09f
		Project 0, id 8a7f0c8d-2e99-40ce-8209-11a91f577177
	record 0, id 47e6bba3-76f1-41dc-93bb-1a4f0e1da75c
		Project 0, id feb21887-2f05-462f-b1af-760a358e7ca0
Saved response to /home/silvhua/repositories/notion/data/parsed_test_time_tracking_data.json


# *End of Page*